In [1]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import warnings
import json
warnings.filterwarnings("ignore")
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

## Data Gathering

In [2]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

combined_dfs = {}

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

count_codes = [['AUDUSD.a','AU'], ['USDCAD.a', 'CA'],
               ['USDCHF.a','CH'], ['GBPUSD.a', 'GB'],
               ['USDJPY.a', 'JP'], ['NZDUSD.a', 'NZ'], 
               ['EURUSD.a', 'XM']]

for country in count_codes:

    # Define the endpoint parameters
    flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
    key = f'D.N.N.{country[1]}'
    start_period = '2000'  # Example: Start year 2000
    end_period = '2023'  # Example: End year 2020
    detail = 'full'  # Example: All data and documentation

    # Construct the endpoint URL
    endpoint_url = f'{base_url}/data/{flow}/{key}/all'

    # Define the query parameters
    query_params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': detail
    }

    # Make the GET request
    response = requests.get(endpoint_url, params=query_params)

    # Check for a successful response
    if response.status_code == 200:
        # Assign the text of the response to xml_data
        xml_data = response.text

        # Parse the XML data
        root = ET.fromstring(xml_data)

        # Initialize empty lists to store the data
        time_periods = []
        obs_values = []

        # Iterate through the XML and extract the desired information
        for obs in root.findall(".//Obs"):
            time_period = obs.get('TIME_PERIOD')
            obs_value = obs.get('OBS_VALUE')
            time_periods.append(time_period)
            obs_values.append(obs_value)

        # Create a DataFrame
        df = pd.DataFrame({
            'Time_Period': time_periods,
            'OBS_Value': obs_values
        })

        df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
        # Drop rows with NaN values
        df.dropna(subset=['OBS_Value'], inplace=True)
        df['OBS_Value'] = df['OBS_Value'].astype(float)
        df['Time_Period'] = pd.to_datetime(df['Time_Period'])
        
        df = df.set_index('Time_Period')
        print(f"Getting {country[0]}'s rates")
        rates = get_rates(country[0], mt5.TIMEFRAME_D1, 2500)
        
        combined = pd.concat([df[-len(rates):], rates['close']], join = 'outer', axis = 1)
        
        combined_dfs[country[1]] = combined.dropna()

    else:
        print(f'Failed to retrieve data: {response.status_code}')

Getting AUDUSD.a's rates
Getting USDCAD.a's rates
Getting USDCHF.a's rates
Getting GBPUSD.a's rates
Getting USDJPY.a's rates
Getting NZDUSD.a's rates
Getting EURUSD.a's rates


In [3]:
# Compute Expected Returns
weekly_dfs = {}
features = ['OBS_Value', 'close']

for name, dfs in combined_dfs.items():
    df = combined_dfs[name]
    # Assuming 'df' is your DataFrame
    df['date'] = pd.to_datetime(df.index)
    df.set_index('date', inplace=True)

    # Resample to get the last value of each week
    weekly = df.resample('W').last()
    weekly_dfs[name] = weekly

for df in weekly_dfs.values():
    df['EER_ret'] = df['OBS_Value'].pct_change()

In [4]:
for df in weekly_dfs.values():
    print(df['EER_ret'].iloc[-1])

-0.0017504619274529842
0.0004950985246066075
0.0007063393960797804
-0.0017371163867979655
-0.006657963446475068
0.0007866273352998832
0.0029310693349504646


In [5]:
last_update = {}

for name, df in weekly_dfs.items():
    last_update[name] = df['EER_ret'].iloc[-1]

In [6]:
last_update.keys()
symbols = ['AUDUSD.a', 'USDCAD.a', 'USDCHF.a', 'GBPUSD.a', 'USDJPY.a', 'NZDUSD.a', 'EURUSD.a']

last_update = {symbols[i]: value for i, (key, value) in enumerate(last_update.items())}

## Order Sending / Closing Logic

In [7]:
def close_all():
    close_positions = []
    open_positions = mt5.positions_get()
    open_positions
    for i in open_positions:
        close_positions.append(i)
        
    for pos in close_positions:
        close_position(pos)
        
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'Regres Close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)

In [26]:
close_all()

In [9]:
def send_order(symbol, side, lot, comment):
    
    if side.lower() == 'sell':
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    elif side.lower() == 'buy':
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 5,
        "magic": 234000,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    result

In [8]:
for symbol, value in last_update.items():
    if symbol[:3] == 'USD':
        print(f"Looping through {symbol}")
        if value > 0:
            send_order(symbol, 'sell', 1.00, 'EER')
            print(f"Selling {symbol}")
        else:
            send_order(symbol, 'buy', 1.00, 'EER')
    else:
        if value > 0:
            send_order(symbol, 'buy', 1.00, 'EER')
        else:
            send_order(symbol, 'sell', 1.00, 'EER')

NameError: name 'send_order' is not defined

# Risk Management

### EER / Close return difference forcast (XGBoost Model)
Forecast with an XGBoost model what the difference of returns will likely be between the EER value / currency close price. After finding the likely difference in returns, either increase (rare cases I reckon) position size or decrease accordingly. Allocate 40% of funds to this strategy to this forecast.

### EER Forecast (ARIMA Model)

Forecast next EER value fo each currency for the next week. If EER forecast is within the IQR ranges associated with each currency, adjust lot sizes for the other two strategies accordingly. 

# Orders 

### Simple Order Send
Copying EER return direction as data is released.

### Pair / Triplet Trading

Trading based on a cointegrating relationship within EER values

### Mean Reversion / Momentum 

Based on mean-reverting nature of EER values balanced with the momentum of EER values

# Research

Analyzing the relationship of EER data and close prices. 

In [9]:
for df in weekly_dfs.values():
    df['ret'] = df['close'].pct_change()
    df['ret_diff'] = df['EER_ret'] - df['ret']
    df = df.dropna()

In [12]:
for name, df in weekly_dfs.items():
    print(f"EER Correlation with {name} is: {df['EER_ret'].corr(df['ret'])}")

EER Correlation with AU is: 0.7901864448535375
EER Correlation with CA is: -0.8028662382263347
EER Correlation with CH is: -0.6972064531265869
EER Correlation with GB is: 0.6872038876573432
EER Correlation with JP is: -0.8162789646117149
EER Correlation with NZ is: 0.7759010905793639
EER Correlation with XM is: 0.75461984964802


In [13]:
desc_stats = pd.DataFrame()

for name in weekly_dfs:
    desc_stats[f"{name} Stats"] = weekly_dfs[name]['ret_diff'].describe()
desc_stats

,AU Stats,CA Stats,CH Stats,GB Stats,JP Stats,NZ Stats,XM Stats
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.000331,-0.000780,0.000455,0.000355,-0.001187,0.000385,0.000303
std,0.009016,0.018449,0.020908,0.009577,0.022378,0.009825,0.007543
min,-0.053037,-0.064334,-0.104875,-0.050289,-0.100707,-0.055615,-0.033891
25%,-0.005043,-0.011253,-0.009497,-0.005518,-0.013587,-0.004930,-0.004470
50%,0.000351,-0.000383,-0.000585,0.000610,-0.002483,0.000214,0.000593
75%,0.005571,0.011458,0.009036,0.005837,0.010900,0.005794,0.004755
max,0.033373,0.059054,0.284139,0.036804,0.077664,0.039223,0.029442


In [77]:
eer_vals = pd.DataFrame()

for name in weekly_dfs:
    eer_vals[f'{name}'] = weekly_dfs[name]['OBS_Value']

In [33]:
from statsmodels.tsa.stattools import coint
from itertools import combinations
import pandas as pd
from statsmodels.tsa.stattools import adfuller 

def cointegration_test(series1, series2):
    '''Runs cointegration test on two series'''
    score, p_value, _ = coint(series1, series2)
    return {'Cointegration Score': score, 'p-value': p_value}

In [79]:
all_pairs = list(combinations(eer_vals.columns, 2))
all_trips = list(combinations(eer_vals.columns, 3))

In [83]:
def adf_test(series1, series2):
    '''Runs ADF test on a spread series'''
    spread = series1 - series2
    result = adfuller(spread)
    return {'ADF Statistic': result[0], 'p-value': result[1], 'Critical Values': result[4]}

def johansen_test(data, det_order=0, k_ar_diff=1):

    result = coint_johansen(data, det_order, k_ar_diff)
    return result


In [82]:
# Perform cointegration test on each pair
coint_results = {}
for pair in all_pairs:
    series1, series2 = eer_vals[pair[0]], eer_vals[pair[1]]
    coint_test_result = adf_test(series1, series2)
    if coint_test_result['Critical Values']['10%'] > coint_test_result['ADF Statistic']:
        # print(f"{coint_test_result['Critical Values']['10%']} is smaller than {coint_test_result['ADF Statistic']}. Adding to results")
        coint_results[pair] = {'ADF_Stat': round(coint_test_result['ADF Statistic'], 4), 
                               '10%_stat': round(coint_test_result['Critical Values']['10%'], 4)}
coint_results

{('AU', 'NZ'): {'ADF_Stat': -4.1766, '10%_stat': -2.5699},
 ('CA', 'NZ'): {'ADF_Stat': -3.5186, '10%_stat': -2.5699},
 ('CA', 'XM'): {'ADF_Stat': -2.6632, '10%_stat': -2.5699},
 ('NZ', 'XM'): {'ADF_Stat': -2.6018, '10%_stat': -2.5699}}

In [108]:
for sym in all_trips:
    data = eer_vals[sym[0]], eer_vals[sym[1]], eer_vals[sym[2]]
test_result = johansen_test(pd.DataFrame(data).T)
n = len(data)
for i in range(n):
    print(f"  At 90%: {test_result.trace_stat_crit_vals[i, 0]}, Trace Stat = {test_result.trace_stat[i]}")
    print(f"  At 95%: {test_result.trace_stat_crit_vals[i, 1]}, Trace Stat = {test_result.trace_stat[i]}")
    print(f"  At 99%: {test_result.trace_stat_crit_vals[i, 2]}, Trace Stat = {test_result.trace_stat[i]}")

  At 90%: 27.0669, Trace Stat = 26.334654627092632
  At 95%: 29.7961, Trace Stat = 26.334654627092632
  At 99%: 35.4628, Trace Stat = 26.334654627092632
  At 90%: 13.4294, Trace Stat = 7.676091331107257
  At 95%: 15.4943, Trace Stat = 7.676091331107257
  At 99%: 19.9349, Trace Stat = 7.676091331107257
  At 90%: 2.7055, Trace Stat = 0.7471535183749242
  At 95%: 3.8415, Trace Stat = 0.7471535183749242
  At 99%: 6.6349, Trace Stat = 0.7471535183749242


In [115]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
trip_coint_res = {}
for sym in all_trips:
    data = eer_vals[sym[0]], eer_vals[sym[1]], eer_vals[sym[2]]
    test_result = johansen_test(pd.DataFrame(data).T)
    
    if test_result.trace_stat_crit_vals[1,0] < test_result.trace_stat[1]:
        print(f"Result: {test_result.trace_stat_crit_vals[1,0]} < {test_result.trace_stat[1]}")
        print(f"{sym} cointegrates on a trip. level")
    
    # result = johansen_test(data)

Result: 13.4294 < 13.937368787819146
('AU', 'CA', 'GB') cointegrates on a trip. level
Result: 13.4294 < 19.290192302491445
('AU', 'CA', 'NZ') cointegrates on a trip. level
Result: 13.4294 < 17.71064339528753
('AU', 'CA', 'XM') cointegrates on a trip. level
Result: 13.4294 < 13.440846845315779
('AU', 'GB', 'XM') cointegrates on a trip. level
Result: 13.4294 < 18.203896258830596
('AU', 'NZ', 'XM') cointegrates on a trip. level
Result: 13.4294 < 15.570874597943478
('CA', 'CH', 'JP') cointegrates on a trip. level
Result: 13.4294 < 16.647381123213624
('CA', 'GB', 'NZ') cointegrates on a trip. level
Result: 13.4294 < 16.29832999361456
('CA', 'GB', 'XM') cointegrates on a trip. level
Result: 13.4294 < 23.233955281631843
('CA', 'NZ', 'XM') cointegrates on a trip. level
Result: 13.4294 < 14.135697301903793
('GB', 'NZ', 'XM') cointegrates on a trip. level


In [119]:
trip_coint_res = {}
for sym in all_trips:
    data = pd.DataFrame({sym[0]: eer_vals[sym[0]], sym[1]: eer_vals[sym[1]], sym[2]: eer_vals[sym[2]]})
    test_result = johansen_test(data)
    
    
    cointegrating_relations = sum(test_result.trace_stat > test_result.trace_stat_crit_vals[:, 1])
    if cointegrating_relations >= 3:
        trip_coint_res[sym] = cointegrating_relations

    # print(f"{sym} shows evidence of {cointegrating_relations} cointegrating relationship(s).")

# Output results
for trip, relations in trip_coint_res.items():
    print(f"{trip} has {relations} cointegrating relationship(s).")

('AU', 'CA', 'NZ') has 3 cointegrating relationship(s).
('AU', 'CA', 'XM') has 3 cointegrating relationship(s).
('AU', 'NZ', 'XM') has 3 cointegrating relationship(s).
('CA', 'GB', 'NZ') has 3 cointegrating relationship(s).
('CA', 'GB', 'XM') has 3 cointegrating relationship(s).
('CA', 'NZ', 'XM') has 3 cointegrating relationship(s).


In [124]:
eer_vals[['AU', 'CA', 'NZ']].describe()

,AU,CA,NZ
count,501.000000,501.000000,501.000000
mean,106.988363,104.074631,104.420279
std,5.515604,4.818668,4.175869
min,89.110000,93.230000,95.260000
25%,103.110000,101.220000,101.550000
50%,106.460000,102.940000,103.740000
75%,110.230000,105.950000,106.510000
max,124.420000,121.550000,116.580000


## Risk Management Framework

Aim is to take into account associated statistics between the two and minimise volatility whilst maximising return. Appropriate measures can include:
- MVO or BL Portfolio
- Correlation analysis and appropriate hedging associated 